In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/content/churnDriverTopics_prod_2111.csv')

In [4]:
data

,clusterId,clusterName,topicName,topicId
0,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt conversation termination,398b4aee-75f0-11ef-8a13-822886e55f82
1,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt conversation ending,45462704-75f1-11ef-9026-321b9d3e98d2
2,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt call ending,52ce6b86-7603-11ef-b5e3-22b1dd5eca87
3,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt call termination,768eeb6c-7604-11ef-8a13-822886e55f82
4,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt ending of conversation,b3fb4a16-7610-11ef-8c04-b230aff9aa17
...,...,...,...,...
160208,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,preference for local store support,07d9e54a-8816-11ef-ac35-1e32fdf09a14
160209,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,in store availability preference,496bc78c-8855-11ef-b9fe-3e04455d968a
160210,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,positive in store experience expectation,1356c682-8874-11ef-9f2b-6a273c902a20
160211,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,preference for in store delivery,468d61fe-8875-11ef-9e66-7eb78fb7d69b


In [5]:
# !pip install transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 10.7 MB/s eta 0:00:00


In [6]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np

# Step 1: Load the dataset
file_path = '/content/churnDriverTopics_prod_2111.csv'
df = pd.read_csv(file_path)

# Step 2: Preprocessing the 'topicName' column
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['cleaned_topicName'] = df['topicName'].apply(preprocess_text)

# Step 3: Generate BERT embeddings for the cleaned text
model = SentenceTransformer('bert-base-nli-mean-tokens')
embeddings = model.encode(df['cleaned_topicName'].tolist())

# Step 4: Clustering using KMeans
n_clusters = df['clusterId'].nunique()
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster_labels'] = kmeans.fit_predict(embeddings)

# Step 5: Cluster quality metrics

# Silhouette Score
sil_score = silhouette_score(embeddings, df['cluster_labels'])
print(f'Silhouette Score: {sil_score}')

# Davies-Bouldin Index
db_index = davies_bouldin_score(embeddings, df['cluster_labels'])
print(f'Davies-Bouldin Index: {db_index}')

# Step 6: Intra-cluster similarity
def intra_cluster_similarity(df, embeddings):
    intra_sim = {}
    for cluster_id in df['clusterId'].unique():
        cluster_embeddings = embeddings[df['clusterId'] == cluster_id]
        if len(cluster_embeddings) > 1:
            similarity = cosine_similarity(cluster_embeddings)
            avg_similarity = np.mean(similarity[np.triu_indices_from(similarity, 1)])
        else:
            avg_similarity = 1.0
        intra_sim[cluster_id] = avg_similarity
    return intra_sim

intra_sim = intra_cluster_similarity(df, embeddings)
for cluster_id, sim in intra_sim.items():
    print(f'Intra-cluster Similarity for {cluster_id}: {sim}')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Silhouette Score: 0.0669572576880455
Davies-Bouldin Index: 2.497174597607922
Intra-cluster Similarity for 398b4aee-75f0-11ef-8a13-822886e55f82: 0.8394845724105835
Intra-cluster Similarity for 5fdf2760-75dc-11ef-99c2-b6ba65adfbe8: 0.6271739602088928
Intra-cluster Similarity for 64f8083c-75e8-11ef-a976-86481ced461b: 0.6806303262710571
Intra-cluster Similarity for d7f5c960-75fb-11ef-9026-321b9d3e98d2: 0.6451415419578552
Intra-cluster Similarity for f1d3dc9a-75de-11ef-b5e3-22b1dd5eca87: 0.6297609806060791
Intra-cluster Similarity for 4ab65a2a-75dc-11ef-a00f-8ad760bdae2f: 0.6495228409767151
Intra-cluster Similarity for abc8d30e-75de-11ef-a00f-8ad760bdae2f: 0.7827185988426208
Intra-cluster Similarity for 8eafd69c-75dd-11ef-9970-8e75b8507098: 0.7242575287818909
Intra-cluster Similarity for 3c54681c-7921-11ef-aa1a-26c508c6cd87: 0.8796094059944153
Intra-cluster Similarity for edb93a02-75dd-11ef-9026-321b9d3e98d2: 0.6403895020484924
Intra-cluster Similarity for aef8ef8a-75e0-11ef-a976-86481ced46

In [7]:
df

,clusterId,clusterName,topicName,topicId,cleaned_topicName,cluster_labels
0,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt conversation termination,398b4aee-75f0-11ef-8a13-822886e55f82,abrupt conversation termination,468
1,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt conversation ending,45462704-75f1-11ef-9026-321b9d3e98d2,abrupt conversation ending,731
2,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt call ending,52ce6b86-7603-11ef-b5e3-22b1dd5eca87,abrupt call ending,468
3,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt call termination,768eeb6c-7604-11ef-8a13-822886e55f82,abrupt call termination,468
4,398b4aee-75f0-11ef-8a13-822886e55f82,Abrupt Conversation Termination,abrupt ending of conversation,b3fb4a16-7610-11ef-8c04-b230aff9aa17,abrupt ending of conversation,731
...,...,...,...,...,...,...
160208,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,preference for local store support,07d9e54a-8816-11ef-ac35-1e32fdf09a14,preference for local store support,879
160209,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,in store availability preference,496bc78c-8855-11ef-b9fe-3e04455d968a,in store availability preference,879
160210,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,positive in store experience expectation,1356c682-8874-11ef-9f2b-6a273c902a20,positive in store experience expectation,879
160211,eeb983c2-75dc-11ef-8c04-b230aff9aa17,Willingness to Visit Store for Resolution,preference for in store delivery,468d61fe-8875-11ef-9e66-7eb78fb7d69b,preference for in store delivery,879
